# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

In [5]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs.  
**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

In [6]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

**Note:** This and all other lab notebooks of this course use OpenAI library version `0.27.0`. 

In order to use the OpenAI library version `1.0.0`, here is the code that you would use instead for the `get_completion` function:

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

#### Tactic 2: Ask for a structured output
- JSON, HTML

In [7]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

{
  "books": [
    {
      "book_id": 1,
      "title": "The Enigma of Elysium",
      "author": "Evelyn Sinclair",
      "genre": "Mystery"
    },
    {
      "book_id": 2,
      "title": "Whispers in the Wind",
      "author": "Nathaniel Blackwood",
      "genre": "Fantasy"
    },
    {
      "book_id": 3,
      "title": "Echoes of the Past",
      "author": "Amelia Hart",
      "genre": "Romance"
    }
  ]
}


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [8]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [9]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [10]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the unwavering determination to rise again after every fall, and the ability to find strength in the face of adversity. Just as a diamond is formed under immense pressure, resilience is forged through challenges and hardships, making us stronger and more resilient in the process.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [11]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and remain undeterred in their adventurous spirits.

2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.

3 - Jack, Jill

4 - {
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.",
  "num_names": 2
}


#### Ask for output in a specified format

In [12]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, siblings from a charming village, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and continue exploring with delight.

Translation: Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, mais ils rentrent chez eux et continuent à explorer avec joie.

Names: Jack, Jill

Output JSON: {"french_summary": "Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, mais ils rentrent chez eux et continuent à explorer avec joie.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [13]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost as a function of the number of square feet.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [14]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

1. Land cost: $100 / square foot
The cost of land is $100 multiplied by the size of the installation in square feet.

2. Solar panel cost: $250 / square foot
The cost of solar panels is $250 multiplied by the size of the installation in square feet.

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat fee of $100,000 per year, plus $10 multiplied by the size of the installation in square feet.

Total cost: Land cost + Solar panel cost + Maintenance cost

Let's calculate the total cost using the actual solution:

Total cost = (100 * x) + (250 * x) + (100,000 + (10 * x))
           = 100x + 250x + 100,000 + 10x
           = 360x + 100,000

Is the student's solution the same as the actual solution just calculated:
No

Student grade:
Incorrect


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [15]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced toothbrush designed to provide a superior brushing experience. Boie is a company known for its innovative oral care products, and the AeroGlide UltraSlim Smart Toothbrush is no exception.

One of the standout features of this toothbrush is its ultra-slim design. The brush head is only 2mm thick, making it much thinner than traditional toothbrushes. This slim profile allows for better access to hard-to-reach areas of the mouth, ensuring a thorough and effective clean.

The AeroGlide UltraSlim Smart Toothbrush also incorporates smart technology. It connects to a mobile app via Bluetooth, allowing users to track their brushing habits and receive personalized recommendations for improving their oral hygiene routine. The app provides real-time feedback on brushing technique, duration, and coverage, helping users to achieve optimal oral health.

The toothbrush features soft, antimicrobial bristles made from a dura

## Try experimenting on your own!

In [16]:
# Summarization
text = f"""
Hello, my name is Jacob Miesner. \
I am a Data Scientist and Technology Leader. \
I started in the field of Data Science in \
2018 and have become obsessed with it ever \
since. There is nothing quite like being \
passionate about a subject where there \
is no shortage of information to consume, \
new concepts to learn, and boundaries to \
be pushed. The feeling of taking an idea \
from inception to deployment is unmatched.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
< >{text}< >
"""
response = get_completion(prompt)
print(response)

Jacob Miesner is a Data Scientist and Technology Leader who became obsessed with the field of Data Science in 2018 and finds joy in continuously learning and pushing boundaries, particularly in taking ideas from inception to deployment.


In [17]:
# JSON
prompt = f"""
Generate a list of four Machine Learning concepts, \
each with a name, one sentence description, an \
integer indicating its level of diffulty on a scale of 1-10.
Provide them in JSON format with the following keys: 
name, description, difficulty.
"""
response = get_completion(prompt)
print(response)

{
  "concepts": [
    {
      "name": "Linear Regression",
      "description": "A statistical model that predicts a continuous outcome variable based on one or more predictor variables.",
      "difficulty": 4
    },
    {
      "name": "Convolutional Neural Networks",
      "description": "Deep learning models designed for analyzing visual data, often used in image recognition tasks.",
      "difficulty": 8
    },
    {
      "name": "Reinforcement Learning",
      "description": "A type of machine learning where an agent learns to make decisions in an environment to maximize a reward signal.",
      "difficulty": 7
    },
    {
      "name": "Support Vector Machines",
      "description": "A supervised learning algorithm used for classification and regression analysis.",
      "difficulty": 6
    }
  ]
}


In [18]:
# Check if conditions are met (Success)
text_1 = f"""
Once up on a time there was a sea snail. \
He crawled the ocean floor searching for \
food and shelter. His shelter was his shell. \
Without his shell he would be exposed to \
danger.
"""
prompt = f"""
You will be provided with text delimited by three dashes. 
If the text contains information about an animal then \
the information in the following format.

Animal:
Habitat:
Land/Sea:

If the text does not contain information about an animal, \ 
then simply write \"No animal information.\"

---{text_1}---
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Animal: Sea snail
Habitat: Ocean floor
Land/Sea: Sea


In [19]:
# Check if conditions are met (Failure)
text_2 = f"""
Playing the game of baseball is \
a difficult endeavor. It is said \
that hitting a baseball is the \
hardest thing to do in sports. This \
is because of the speed of the ball, \
the movement of the ball, and the \
focus and reflexes required to hit \
the ball on a consistent basis.
"""
prompt = f"""
You will be provided with text delimited by three dashes. 
If the text contains information about an animal then \
the information in the following format.

Animal:
Habitat:
Land/Sea:

If the text does not contain information about an animal, \ 
then simply write \"No animal information.\"

---{text_1}---
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
Animal: Sea snail
Habitat: Ocean floor
Land/Sea: Sea


In [20]:
# Few Shot Prompting
prompt = f"""
Your task is to answer in a consistent style.

Informal: Hey, can u update me on the project?
Formal: Hello, could you please provide me with an update on the project?
Informal: lol, that's way too expensive, dude.
Formal: That appears to be excessively expensive.
Informal: btw, can't make it to the meeting tomorrow.
Formal: By the way, I will not be able to attend the meeting tomorrow.
Informal: Dude, this Deeplearning.ai short course is sicckkkk!!!
Formal:"""
response = get_completion(prompt)
print(response)

This Deeplearning.ai short course is highly impressive.


In [21]:
# Specify Steps in a task
text = f"""
In an enchanted forest, a wise old owl named Oliver \ 
taught curious animals about the mysteries of the \ 
world. Every evening, creatures of all sizes gathered \ 
to listen to Oliver's tales under the moonlit sky. \ 
One night, a mischievous fox named Felix played a prank \ 
that startled everyone, but Oliver, with his gentle wisdom, \ 
turned it into a lesson about kindness and understanding. \ 
The animals left with a newfound appreciation for their \ 
forest community.
"""

# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Identify the main theme of the story in the text delimited by triple \
backticks.
2 - List the moral lessons mentioned in the rewritten story.
3 - Output a json object that contains the following \
keys: theme, moral_lessons.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - The main theme of the story is the importance of kindness and understanding in a community.

2 - The moral lessons mentioned in the rewritten story are:
- Kindness and understanding can turn negative situations into positive learning experiences.
- Appreciating and valuing one's community is important.

3 - {
  "theme": "importance of kindness and understanding in a community",
  "moral_lessons": [
    "Kindness and understanding can turn negative situations into positive learning experiences",
    "Appreciating and valuing one's community is important"
  ]
}


In [22]:
# Ask for a specified output format
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Identify the main theme of the story in the text delimited by triple \
backticks.
2 - List the moral lessons mentioned in the rewritten story.
3 - Output a json object that contains the following \

keys: theme, moral_lessons.
Use the following format:
Text: <text to summarize>
Theme: <theme>
Moral Lessions: <moral Lessons>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Theme: The theme of the story is the importance of kindness and understanding in a community.

Moral Lessons: 
1. Kindness: The story teaches the moral lesson of kindness by showing how Oliver, the wise old owl, turns a prank into a lesson about kindness and understanding. This teaches the animals the importance of being kind to one another.
2. Understanding: The story also emphasizes the moral lesson of understanding. Oliver helps the animals understand the importance of their forest community and the need to appreciate and support each other.

Output JSON: 
{
  "theme": "The importance of kindness and understanding in a community",
  "moral_lessons": [
    "Kindness",
    "Understanding"
  ]
}


In [23]:
# Ask model to reason about solution
prompt = f"""
Your task is to determine if the given scenario's outcome is accurate or not.
To solve the problem, do the following:
- First, work out your own solution to the problem, including the final outcome.
- Then compare your solution to the outcome provided in the scenario and evaluate if it is accurate.
Do not decide on the accuracy of the scenario's outcome until you have solved the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Scenario:
A small bookstore sells novels, magazines, and newspapers. 
- Novels sell for $15 each
- Magazines sell for $5 each
- Newspapers sell for $2 each
If a customer buys 3 novels, 5 magazines, and 2 newspapers, what is the total cost?

Given Outcome:
```
The total cost for the customer is $74.
```
Actual Solution:
"""
response = get_completion(prompt)
print(response)

Question:
What is the total cost for the customer?

Student's solution:
To find the total cost, we need to calculate the cost of each item and then add them together. 

The cost of 3 novels is 3 * $15 = $45.
The cost of 5 magazines is 5 * $5 = $25.
The cost of 2 newspapers is 2 * $2 = $4.

Therefore, the total cost for the customer is $45 + $25 + $4 = $74.

Actual Solution:
To find the total cost, we need to calculate the cost of each item and then add them together. 

The cost of 3 novels is 3 * $15 = $45.
The cost of 5 magazines is 5 * $5 = $25.
The cost of 2 newspapers is 2 * $2 = $4.

Therefore, the total cost for the customer is $45 + $25 + $4 = $74.

Is the student's solution the same as actual solution just calculated:
Yes

Student grade:
Correct


In [24]:
# Model Hallucination
prompt = f"""
Tell me about SuperStride SmartX Pro Scooter by Razor.
"""
response = get_completion(prompt)
print(response)

The SuperStride SmartX Pro Scooter is a high-performance scooter designed and manufactured by Razor, a well-known brand in the scooter industry. It is a top-of-the-line model that offers advanced features and superior performance for riders.

One of the standout features of the SuperStride SmartX Pro Scooter is its smart technology integration. It comes equipped with a built-in LED display that provides real-time feedback on speed, distance traveled, battery life, and other important metrics. This allows riders to track their performance and make adjustments as needed.

The scooter is powered by a high-torque, brushless hub motor that provides a smooth and powerful ride. It has a top speed of up to 15 mph (24 km/h), making it suitable for both commuting and recreational use. The motor is also whisper-quiet, ensuring a noise-free riding experience.

The SuperStride SmartX Pro Scooter features a durable and lightweight aluminum frame, making it easy to maneuver and transport. It has a la

#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.